# Imports

In [86]:
import os
import json
import requests
from dotenv import load_dotenv

from gigachat import GigaChat

# Data

In [5]:
DATA = "../data/raw/data.json"

with open(DATA, "r", encoding="utf-8") as file:
    data = json.load(file)

data[0]

{'id': '7DBD44',
 'task': 'Выберите один или несколько правильных ответов.\nУкажите варианты ответов, в которых в обоих словах одного ряда пропущена одна и та же буква. Запишите номера ответов.\nнедолюбл..вать, завлад..вать\nизменч..вость, варень..це\nвыносл..вый, рассматр..вая\nотрасл..вой, гречн..вый\nсветл..нький, медвед..ца\n',
 'answer': '',
 'tags': []}

In [85]:
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
GIGACHAT_API_KEY = os.getenv("GIGACHAT_API_KEY")

In [ ]:
def call_mistral(query: str) -> str:
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json",
    }

    # Define the payload with the user query
    payload = {
        "messages": [
            {
                "role": "system",
                "content": """
                    Вы студент, который отлично решает экзаменационные задачи.
                    Следуйте инструкциям, указанным в задании. Формат ответа
                    строгий и описан в тексте.
                    Не пишите ничего, кроме ответа. Ответ должен быть на русском языке.
                    Не пишите пояснение к ответу. Любой текст, кроме ответа
                    будет считаться неправильным ответом. Ваша задача — набрать
                    максимум баллов
                """,
            },
            {"role": "user", "content": query},
        ],
        "model": "mistral-large-latest",
        "temperature": 0.8,
        "top_p": 0.95,
    }

    try:
        # Make the API request
        response = requests.post(
            "https://api.mistral.ai/v1/chat/completions", headers=headers, json=payload
        )

        # Check if the request was successful
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"]
        else:
            response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


giga = GigaChat(
    credentials=GIGACHAT_API_KEY,
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)


def call_gigachat(query: str) -> str:
    return (
        giga.chat(
            {
                "messages": [
                    {
                        "role": "system",
                        "content": """
                            Вы студент, который отлично решает экзаменационные задачи по русскому языку.
                            Следуйте инструкциям, указанным в задании. Формат ответа строгий и описан в тексте.
                            Если 
                        """,
                    },
                    {"role": "user", "content": query},
                ],
                "temperature": 0.1,
                "top_p": 1,
            }
        )
        .choices[0]
        .message.content
    )

# Experiments

In [128]:
task = data[566]["task"]
print(task)
response = call_gigachat(task)
response

Выберите один или несколько правильных ответов.
Укажите варианты ответов, в которых в обоих словах одного ряда пропущена одна и та же буква. Запишите номера ответов.
подразум..вая, налаж..вать
алюмини..вый, затм..вать
раскра..вать, болот..стая (местность)
милост..вый, (улицы) обезлюд..ли
гру́ш..вый (сок), берест..ной 



'13'

In [ ]:
call_gigachat("")

'Для решения этой задачи воспользуемся формулой площади равностороннего треугольника через сторону и высоту:\n\n$$S = \\frac{1}{2}ah,$$\n\nгде $a$ - сторона треугольника, а $h$ - высота.\n\nПо условию задачи высота $h$ равна 3. Сторона равностороннего треугольника вычисляется по формуле:\n\n$$a = \\frac{2h}{\\sqrt{3}},$$\n\nподставляем значение высоты:\n\n$$a = \\frac{2*3}{\\sqrt{3}} = \\frac{6}{\\sqrt{3}}.$$\n\nУпростим дробь, умножив числитель и знаменатель на $\\sqrt{3}$:\n\n$$a = \\frac{6\\sqrt{3}}{3} = 2\\sqrt{3}.$$\n\nТеперь подставим значения стороны и высоты в формулу площади:\n\n$$S = \\frac{1}{2} * 2\\sqrt{3} * 3 = \\sqrt{3} * 3 = 3\\sqrt{3}.$$'